In [1]:
import sys

!which {sys.executable}

/ut3/jerome/miniconda3/envs/wakai/bin/python


In [2]:
import sys
sys.path.insert(0, '../code')

# TOI catalog

In [3]:
from catalog import get_tois

tois = get_tois(clobber=True)
# tois.columns.tolist()

Saved:  /ut3/jerome/github/research/project/wakai/data/TOIs.csv


In [4]:
tois.tail()

,TIC ID,TOI,Previous CTOI,Master,SG1A,SG1B,SG2,SG3,SG4,SG5,...,Stellar Mass (M_Sun) err,Sectors,Date TOI Alerted (UTC),Date TOI Updated (UTC),Date Modified,Comments,ra_deg,dec_deg,GaiaDR3_exofop,GaiaDR3_vizier
6383,352301423,7334.01,NaN,3,4,3,3,1,4,4,...,NaN,"14,15,16,17,40,41,47,50,54,55,56,57,60,74,75,7...",2025-03-06,2025-03-06,2025-03-11 12:05:04,found in faint-star QLP search,297.182208,58.384411,NaN,NaN
6384,75417968,7335.01,NaN,3,4,3,3,1,4,4,...,0.235905,"55,81,82",2025-03-06,2025-03-06,2025-03-11 12:04:40,found in faint-star QLP search,308.688917,17.403658,NaN,NaN
6385,271017537,7336.01,NaN,1,4,1,1,1,4,4,...,0.093084,"17,18,57,58,77,78,84,85",2025-03-06,2025-03-06,2025-03-11 12:04:37,slight depth-aperture correlation,353.525833,65.541447,NaN,NaN
6386,323407659,7337.01,NaN,3,4,3,3,1,4,4,...,0.251658,"16,18,24,57,77,78,84,85",2025-03-06,2025-03-06,2025-03-11 12:05:13,multiple stars in pixel; pulsating host; radiu...,333.355333,67.394175,NaN,NaN
6387,309133240,7338.01,TIC 309133240.01,1,4,1,1,2,4,4,...,0.107693,"18,58,85",2025-03-06,2025-03-05,2025-03-11 12:05:17,V-shaped; possible offset; possible odd-even,25.226167,50.300333,NaN,NaN


In [5]:
tois[tois.Comments.apply(lambda x: 'young' in str(x))]

,TIC ID,TOI,Previous CTOI,Master,SG1A,SG1B,SG2,SG3,SG4,SG5,...,Stellar Mass (M_Sun) err,Sectors,Date TOI Alerted (UTC),Date TOI Updated (UTC),Date Modified,Comments,ra_deg,dec_deg,GaiaDR3_exofop,GaiaDR3_vizier
731,146520535,942.01,NaN,3,5,5,3,4,4,4,...,0.097250,5,2019-07-24,2020-05-20,2022-12-14 12:09:24,Probable young K dwarf. Transit is believable ...,76.649625,-20.245614,2.974907e+18,2.974907e+18
2865,330135981,3604.01,NaN,3,4,4,3,2,4,4,...,NaN,"16,17,56,57",2021-06-23,2024-09-06,2024-09-06 00:00:00,V-shaped; found in faint-star QLP search; no s...,333.501042,56.918981,2.198435e+18,2.198435e+18
4848,308300731,5743.01,NaN,3,4,3,3,4,4,4,...,0.109025,"26,53,79,80",2022-09-01,2024-08-25,2024-08-31 12:14:55,radius possibly underestimated; potentially yo...,267.904333,24.427456,4.581034e+18,4.581034e+18
5040,236785891,5956.01,TIC 236785891.01,3,4,4,3,2,4,4,...,0.021842,"56,57",2022-11-16,2023-02-03,2024-09-01 00:00:00,V-shaped; large; young host star; ~5 sigma odd...,283.317250,59.817658,2.155491e+18,2.155491e+18
5326,435878153,6258.01,NaN,2,4,4,2,4,4,4,...,0.020209,"17,57",2023-04-06,2023-04-06,2024-02-23 00:00:00,low SNR; asymmetric transit shape; flaring pos...,13.085375,20.938875,2.789799e+18,2.789799e+18
5743,253864764,6678.01,NaN,1,4,1,1,3,4,4,...,0.092773,67,2023-09-21,2023-09-21,2024-09-12 00:00:00,rotational modulation at 7-8 days may indicate...,283.536292,-36.656858,6.731034e+18,6.731034e+18


In [5]:
tois.shape

(6388, 66)

# cross-match TOI with Gaia DR3 using `astroquery.xmatch`

In [6]:
import astropy.units as u
from astroquery.xmatch import XMatch
import pandas as pd

gaia_table = XMatch.query(
    cat1=open('../data/TOIs.csv'),
    cat2="vizier:I/355/gaiadr3",  # The Vizier identifier for Gaia DR3
    max_distance=1 * u.arcsec,    # Set the maximum matching radius
    colRA1="ra_deg",  # Column name for Right Ascension in TIC results
    colDec1="dec_deg" # Column name for Declination in TIC results
)
gaia_df = gaia_table.to_pandas()

# results to 6237 matches
matches = gaia_df.copy()

# Apply magnitude filtering criteria
# For main-sequence stars, the difference between TESS T and Gaia G is around 0–0.5 magnitudes
# Redder stars might have T - G values closer to 1–2 magnitudes.
# For giants or highly reddened stars, the difference could be larger.
if False:
    # results to 5971 matches
    matches = []
    for i, row in gaia_df.iterrows():
        teff=row['Stellar Eff Temp (K)']
        if teff<3500:
            magnitude_threshold = 2
        elif (teff>3500) and (teff<6000):
            magnitude_threshold = 1
        else:
            magnitude_threshold = 1.5
        if abs(row['Gmag'] - row['TESS Mag']) < magnitude_threshold:
            matches.append(row)
    matches = pd.concat(matches, axis=1).T
if False:
    # results to 5700 matches
    magnitude_threshold = 1
    matches = gaia_df[abs(gaia_df['Gmag'] - gaia_df['TESS Mag']) < magnitude_threshold]
if False:
    # results to 4878 matches
    teff_threshold = 1000
    matches = gaia_df[abs(gaia_df['Teff'] - gaia_df['Stellar Eff Temp (K)']) < teff_threshold]
matches.tail()

Could not import regions, which is required for some of the functionalities of this module.


,angDist,TIC ID,TOI,Previous CTOI,Master,SG1A,SG1B,SG2,SG3,SG4,...,B_AG,E(BP-RP),b_E(BP-RP),B_E(BP-RP),Lib,RAJ2000,DEJ2000,e_RAJ2000,e_DEJ2000,RADEcorJ2000
7620,0.018288,352301423,7334.01,NaN,3,4,3,3,1,4,...,0.1154,0.0546,0.0466,0.0626,MARCS,297.182251,58.384445,1.009054,1.241652,-0.0370
7621,0.043186,75417968,7335.01,NaN,3,4,3,3,1,4,...,0.1434,0.0707,0.0401,0.0771,MARCS,308.688932,17.403665,0.358187,0.385483,0.4412
7622,0.091872,271017537,7336.01,NaN,1,4,1,1,1,4,...,0.0003,0.0000,0.0000,0.0002,MARCS,353.525357,65.540835,0.208975,0.171382,-0.2766
7623,0.018481,323407659,7337.01,NaN,3,4,3,3,1,4,...,0.3155,0.1658,0.1603,0.1703,MARCS,333.355327,67.394162,0.245584,0.216391,-0.1757
7624,0.044777,309133240,7338.01,TIC 309133240.01,1,4,1,1,2,4,...,0.0038,0.0005,0.0001,0.0020,PHOENIX,25.226082,50.300017,0.254575,0.253889,0.0115


In [7]:
matches.shape

(7625, 204)

In [8]:
matches['GaiaDR3'] = matches['DR3Name'].apply(lambda x: int(x.split()[-1]))
matches.to_csv('../data/TOIs_GaiaDR3_xmatch.csv', index=False)

In [9]:
idx = tois.TOI.isin(matches.TOI.unique())
tois[~idx].shape

(15, 66)

In [10]:
tois[~idx].to_csv('../data/TOIs_GaiaDR3_xmatch_missing.csv', index=False)